In [ ]:
# Import all of the packages needed for the notebook
import skimage.io as io
import numpy as np
import matplotlib.pyplot as plt

# Thresholding in color images

Idea: Define statistics of a population of pixels (mean, standard deviation (covariance matrix)).
Measure distance normalized by covariance.

In [ ]:
marin=io.imread('naip_1-1_1n_s_ca041_2005_100109.jpg')
%matplotlib notebook
plt.figure()
plt.imshow(marin)
plt.show()

In [ ]:
forest_roi = marin[950:1201,3050:3200,:] # region of interest, all channels
%matplotlib notebook
plt.figure()
plt.imshow(forest_roi)
plt.title('Forest ROI')
plt.show()

## Covariance matrix

The covariance matrix contains information about the variance (standard deviation squared) of the pixels in the forested region as well as the covariance between colors.
$$
    \underline{\mu} = \begin{bmatrix}\mu_r \\ \mu_g \\ \mu_b \end{bmatrix}
$$

$$\underline{\underline{\sigma}} = \begin{bmatrix}\sigma_r & \sigma_{rg} & \sigma_{rb} \\
\sigma_{gr} & \sigma_g & \sigma_{gb}\\
\sigma_{br} & \sigma_{bg} & \sigma_b
\end{bmatrix}$$

In [ ]:
# Calculate the covariance matrix for all of the pixels in the roi
# The numpy covariance function expects a 3-by-npixel array
pixels = forest_roi.reshape((-1,3)).transpose()
print('pixels shape is ',pixels.shape)
pix_mean = np.mean(pixels,axis=1)
pix_cov_mat = np.cov(pixels)
print('Mean R,G,B: ',pix_mean)
print('cov mat: \n',pix_cov_mat)

In [ ]:
# For each pixel in the original image, calculate the Mahalanobis distance
ci = np.linalg.inv(pix_cov_mat)
# for clarity, this is how we would use 'for' loops to iterate over the columns and rows of the image
# Note that this is incredibly slow. Do not do this!!
#distance_matrix = np.zeros( marin.shape[:-1] )  # Create a blank matrix of zeres with the same number of cols/rows as 'marin'. :-1 removes 3rd dimension
#for i in range(marin.shape[0]):
#    for j in range(marin.shape[1]):
#        d = (marin[i,j,:] - pix_mean).reshape((3,1)) # force numpy to think of distance as a column vector
#        distance_matrix[i,j] = np.sqrt( d.transpose()@(ci@d) )

# This does the same thing but using matrix algebra
# 1. reshape the image to be 3-by-npixel
marin_pixels = marin.reshape((-1,3)).transpose()
# 2. Calculate marin_pixels - pixel mean
pix_mean = pix_mean.reshape((3,1)) # make this a column vector
d = marin_pixels-pix_mean
print(d.shape)
# calculate sqrt(dT*ci*d) for each pixel
# What I do here is first calculate ci@d, which is a 3-by-npixel matrix containing the value of ci*d for each pixel.
# Then I multiply by d, which has the effect of taking the element-wise product at each pixel between d and (ci*d).
# (d*(ci@d)) will be a 3-by-npixel matrix
# I then sum over columns to obtain the pixel-wise values of d^d*ci*d, which is a 1-by-npixel row vector.
distance_matrix = np.sqrt(np.sum(d*(ci@d),axis=0))
# reshape to image dimensions
distance_matrix = distance_matrix.reshape(marin.shape[:-1])

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,3,figsize=(12,4))
ax[0].imshow(marin)
h=ax[1].imshow(distance_matrix,cmap='jet')
box = ax[1].get_position()
ax[2].set_position([box.x0+box.width+.05, box.y0, .05, box.height])
plt.colorbar(h,cax=ax[2])
plt.show()
plt.figure()
plt.hist(distance_matrix.ravel(),50)
plt.grid()
plt.show()

In [ ]:
# Define threshold value in terms of mahalanobis distance. Shade segmented regions:
thresh=7.
forest_mask = distance_matrix <= thresh
marin_tmp = marin.copy()
marin_tmp[forest_mask] = np.array([200,0,0]) # This will tint the forest redder
marin_tmp = np.uint8(marin_tmp)

%matplotlib notebook
fig, ax = plt.subplots(1,2)
ax[0].imshow(marin)
ax[1].imshow(marin_tmp)
fig.show()

In [ ]:
# Note that you can also make an interactive plot in jupyter:
from ipywidgets import interact
thresh=7.
forest_mask = distance_matrix <= thresh
marin_tmp = marin.copy()
marin_tmp[forest_mask] = np.array([200,0,0]) # This will tint the forest redder
marin_tmp = np.uint8(marin_tmp)

%matplotlib notebook
fig, ax = plt.subplots(1,2)
ax[0].imshow(marin)
ax[1].imshow(marin_tmp)

def update(thresh = 7.5):
    marin_tmp = marin.copy()
    forest_mask = distance_matrix <= thresh
    marin_tmp[forest_mask] = np.array([200,0,0]) # This will tint the forest redder
    marin_tmp = np.uint8(marin_tmp)
    ax[1].clear()
    ax[1].imshow(marin_tmp)
    ax[1].set_xlim(ax[0].get_xlim())
    ax[1].set_ylim(ax[0].get_ylim())
    fig.canvas.draw_idle()

interact(update)

# Edge detection

**Motivation**: Up until now, we've worked with images where different regions had different colors and distinct interpretations associated with each color. The human mind can quickly recognize and 'segment' the image below of a bunch of pebbles into distinct pebbles. 

In [ ]:
# Pebbles
from skimage import filters
from skimage import color

pebbles = io.imread('pebbles1.png')
# Convert to greyscale
pebbles_bw = color.rgb2gray(pebbles)
kernel = np.ones((7,7))
pebbles_medfilt = filters.median(pebbles_bw,selem=kernel)
%matplotlib inline
fig,ax = plt.subplots(1,3,sharex=True,sharey=True,figsize=(12,3))
ax[0].imshow(pebbles)
ax[1].imshow(pebbles_bw,cmap='gray')
ax[2].imshow(pebbles_medfilt,cmap='gray')
plt.show()

In [ ]:
from skimage import feature
%matplotlib notebook
pebbles_canny = feature.canny(pebbles_medfilt,sigma=0.0)

fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(pebbles)
ax[1].imshow(pebbles_canny,cmap='gray')
plt.show()

In [ ]:
import cv2 as cv
import numpy as np
filter_size=5
kernel = np.ones((filter_size,filter_size),np.uint8)  # note this is a 5x5 ‘square’ kernel
pebbles_dilated = cv.dilate(1.0*pebbles_canny,kernel,iterations = 1)

fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(pebbles_canny,cmap='gray')
ax[1].imshow(pebbles_dilated,cmap='gray')
plt.show()

In [ ]:
# Skeletonize the image
from skimage import morphology

pebbles_skel = morphology.skeletonize(pebbles_dilated)
fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(pebbles_dilated,cmap='gray')
ax[0].set_title('Dilated')
ax[1].imshow(pebbles_skel,cmap='gray')
ax[1].set_title('Skeletonized')
plt.show()


In [ ]:
# Invert image
from skimage import measure
from skimage.color import label2rgb

pebbles_inv = 1.0-pebbles_skel
# identify  and label connected components
labels = measure.label(pebbles_inv,connectivity=1)
image_label_overlay = label2rgb(labels, image=pebbles)

fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(pebbles_inv,cmap='gray')
ax[0].set_title('Inverted')
ax[1].imshow(image_label_overlay)
ax[1].set_title('Labels')
plt.show()